In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data_science_challenge_samp_18.csv',parse_dates=[1],keep_date_col = True)

In [3]:
data.columns

Index(['cust_id', 'order_date', 'lane_number', 'total_spend',
       'units_purchased'],
      dtype='object')

In [4]:
print(data.shape)
data = data.drop_duplicates()
print(data.shape)

(323437, 5)
(323021, 5)


In [5]:
data.dtypes

cust_id                     int64
order_date         datetime64[ns]
lane_number                 int64
total_spend                object
units_purchased             int64
dtype: object

In [6]:
cust_count = data.cust_id.value_counts().reset_index()
cust_count.columns = ['cust_id','cust_count']

In [7]:
cust_count.head(2)

,cust_id,cust_count
0,5183161534,543
1,4302370736,535


In [8]:
data.order_date.min(),data.order_date.max()

(Timestamp('2015-03-20 00:00:00'), Timestamp('2016-03-27 00:00:00'))

In [9]:
data.head(5)

,cust_id,order_date,lane_number,total_spend,units_purchased
0,4239597436,2015-03-20,3,19.84,5
1,4913827536,2015-03-20,3,11.1,5
2,4913827536,2015-03-20,1,10,1
3,4271866537,2015-03-20,5,13.68,2
4,5316845735,2015-03-20,4,5.29,1


In [10]:
from datetime import date, timedelta
d2 = date(2016, 3, 27)  # start date
d1 = date(2015, 3, 20)  # end date
delta = d2 - d1         # timedelta
all_dates = []
for i in range(delta.days + 8):
    all_dates.append(d1 + timedelta(i))

In [11]:
all_cust_id = data.cust_id.unique()

In [12]:
data_cus_id = []
data_dates = []
for c_id in all_cust_id:
    for d in all_dates:
        data_cus_id.append(c_id)
        data_dates.append(d)

In [13]:
len(data_cus_id), len(data_dates)

(4758309, 4758309)

In [14]:
full_data = pd.DataFrame({'cust_id':data_cus_id,'order_date':data_dates})

In [15]:
full_data.cust_id.value_counts().unique()

array([381])

In [16]:
full_data.dtypes

cust_id        int64
order_date    object
dtype: object

In [17]:
full_data['order_date'] = pd.to_datetime(full_data.order_date,format= '%Y-%m-%d')

In [18]:
full_data.order_date.min(),full_data.order_date.max()

(Timestamp('2015-03-20 00:00:00'), Timestamp('2016-04-03 00:00:00'))

In [19]:
train = full_data.loc[full_data.order_date <= pd.to_datetime('2016-03-27',format ='%Y-%m-%d' )]
test = full_data.loc[full_data.order_date > pd.to_datetime('2016-03-27',format ='%Y-%m-%d' )]

In [20]:
train.cust_id.value_counts().unique(),test.cust_id.value_counts().unique()

(array([374]), array([7]))

In [21]:
test.shape,train.shape,full_data.shape

((87423, 2), (4670886, 2), (4758309, 2))

In [22]:
import gc
del full_data
gc.collect()

37

In [23]:
print(data.shape)
data = data.groupby(['cust_id','order_date']).head(1)
print (data.shape)

(323021, 5)
(283838, 5)


In [24]:
train = train.merge(data, on=['cust_id','order_date'],how='left')

In [25]:
train['target']=0
train.loc[pd.notnull(train.units_purchased ),'target']=1
train = train.drop(['units_purchased','lane_number','total_spend'],axis=1)

In [26]:
# number of one should be same as the number of data observations and it can be verified as below
train.target.sum() == data.shape[0]

True

In [27]:
train.cust_id.value_counts().unique()

array([374])

In [28]:
# create some features based on date
train['month'] = train.order_date.dt.month
train['weekofyear'] = train.order_date.dt.weekofyear
train['is_month_start'] = train.order_date.dt.is_month_start
train['is_month_end'] = train.order_date.dt.is_month_end
train['day'] = train.order_date.dt.day
train['dayofweek'] = train.order_date.dt.dayofweek

test['month'] = test.order_date.dt.month
test['weekofyear'] = test.order_date.dt.weekofyear
test['is_month_start'] = test.order_date.dt.is_month_start
test['is_month_end'] = test.order_date.dt.is_month_end
test['day'] = test.order_date.dt.day
test['dayofweek'] = test.order_date.dt.dayofweek

In [29]:
# one thing i didn't consider(customers visited the store more than once in a day)
# i simply took the first visit of the day(you could use the aggregrated values)
train.columns

Index(['cust_id', 'order_date', 'target', 'month', 'weekofyear',
       'is_month_start', 'is_month_end', 'day', 'dayofweek'],
      dtype='object')

In [30]:
data['month'] = data.order_date.dt.month
data['weekofyear'] = data.order_date.dt.weekofyear
data['is_month_start'] = data.order_date.dt.is_month_start
data['is_month_end'] = data.order_date.dt.is_month_end
data['day'] = data.order_date.dt.day
data['dayofweek'] = data.order_date.dt.day


In [31]:
data['total_spend'] = data.total_spend.str.replace(',', '')
data['total_spend']=data.total_spend.astype(float)

In [32]:
# in next few llines stats based features are created

In [33]:
#k1 = data[['cust_id','total_spend']].groupby('cust_id').agg('mean').reset_index()
#k1.columns = ['cust_id','cust_id_total_send_mean']
#train = train.merge(k1,on='cust_id',how='left')
#test = test.merge(k1,on='cust_id',how='left')

#k2 = data[['cust_id','total_spend']].groupby('cust_id').agg('median').reset_index()
#k2.columns = ['cust_id','cust_id_total_send_median']
#train = train.merge(k2,on='cust_id',how='left')
#test = test.merge(k2,on='cust_id',how='left')

#k3 = data[['cust_id','total_spend']].groupby('cust_id').agg('std').reset_index()
#k3.columns = ['cust_id','cust_id_total_send_std']
#train = train.merge(k3,on='cust_id',how='left')
#test = test.merge(k3,on='cust_id',how='left')

In [34]:
#k1 = data[['cust_id','units_purchased']].groupby('cust_id').agg('mean').reset_index()
#k1.columns = ['cust_id','cust_id_units_purchased_mean']
#train = train.merge(k1,on='cust_id',how='left')
#test = test.merge(k1,on='cust_id',how='left')

#k2 = data[['cust_id','units_purchased']].groupby('cust_id').agg('median').reset_index()
#k2.columns = ['cust_id','cust_id_units_purchased_median']
#train = train.merge(k2,on='cust_id',how='left')
#test = test.merge(k2,on='cust_id',how='left')

#k3 = data[['cust_id','units_purchased']].groupby('cust_id').agg('std').reset_index()
#k3.columns = ['cust_id','cust_id_units_purchased_std']
#train = train.merge(k3,on='cust_id',how='left')
#test = test.merge(k3,on='cust_id',how='left')

In [35]:
k1 = data[['cust_id','month','units_purchased']].groupby(['cust_id','month']).agg('mean').reset_index()
k1.columns = ['cust_id','month','cust_id_month_units_purchased_mean']
train = train.merge(k1,on=['cust_id','month'],how='left')
test = test.merge(k1,on=['cust_id','month'],how='left')

k2 = data[['cust_id','month','units_purchased']].groupby(['cust_id','month']).agg('median').reset_index()
k2.columns = ['cust_id','month','cust_id_month_units_purchased_median']
train = train.merge(k2,on=['cust_id','month'],how='left')
test = test.merge(k2,on=['cust_id','month'],how='left')

k3 = data[['cust_id','month','units_purchased']].groupby(['cust_id','month']).agg('std').reset_index()
k3.columns = ['cust_id','month','cust_id_month_units_purchased_std']
train = train.merge(k3,on=['cust_id','month'],how='left')
test = test.merge(k3,on=['cust_id','month'],how='left')

In [36]:
k1 = data[['cust_id','month','total_spend']].groupby(['cust_id','month']).agg('mean').reset_index()
k1.columns = ['cust_id','month','cust_id_month_total_spend_mean']
train = train.merge(k1,on=['cust_id','month'],how='left')
test = test.merge(k1,on=['cust_id','month'],how='left')

k2 = data[['cust_id','month','total_spend']].groupby(['cust_id','month']).agg('median').reset_index()
k2.columns = ['cust_id','month','cust_id_month_total_spend_median']
train = train.merge(k2,on=['cust_id','month'],how='left')
test = test.merge(k2,on=['cust_id','month'],how='left')

k3 = data[['cust_id','dayofweek','total_spend']].groupby(['cust_id','dayofweek']).agg('std').reset_index()
k3.columns = ['cust_id','dayofweek','cust_id_dayofweek_total_spend_std']
train = train.merge(k3,on=['cust_id','dayofweek'],how='left')
test = test.merge(k3,on=['cust_id','dayofweek'],how='left')

In [37]:
k1 = data[['cust_id','lane_number','total_spend']].groupby(['cust_id','lane_number']).agg('mean').reset_index()
k1 = k1.pivot(index='cust_id', columns='lane_number', values='total_spend').reset_index().replace(np.nan,0)
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')

In [ ]:
k1 = data[['cust_id','month','total_spend']].groupby(['cust_id','month']).agg('mean').reset_index()
k1 = k1.pivot(index='cust_id', columns='month', values='total_spend').reset_index().replace(np.nan,0)
k1 = k1[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
k1.columns = ['cust_id', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6','m7', 'm8', 'm9', 'm10', 'm11']
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')

In [ ]:
k1 = data[['cust_id','month','total_spend']].groupby(['cust_id','month']).agg('mean').reset_index()
k1 = k1.pivot(index='cust_id', columns='month', values='total_spend').reset_index().replace(np.nan,0)
k1 = k1[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
k1.columns = ['cust_id', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6','m7', 'm8', 'm9', 'm10', 'm11']
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')

In [ ]:
k1 = data[['cust_id','month','total_spend']].groupby(['cust_id','month']).agg('count').reset_index()
k1.columns = ['cust_id','month','cust_id_month_count']
train = train.merge(k1,on=['cust_id','month'],how='left')
test = test.merge(k1,on=['cust_id','month'],how='left')


k1 = data[['cust_id','total_spend']].groupby(['cust_id']).agg('count').reset_index()
k1.columns = ['cust_id','cust_id_count']
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')


In [ ]:
k1 = data[['cust_id','is_month_start','total_spend']].groupby(['cust_id','is_month_start']).agg('mean').reset_index()
k1 = k1.pivot(index='cust_id', columns='is_month_start', values='total_spend').reset_index().replace(np.nan,0)
k1.columns = ['cust_id', 'is_month_start0', 'is_month_start1']
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')

In [ ]:
k1 = data[['cust_id','is_month_end','total_spend']].groupby(['cust_id','is_month_end']).agg('mean').reset_index()
k1 = k1.pivot(index='cust_id', columns='is_month_end', values='total_spend').reset_index().replace(np.nan,0)
k1.columns = ['cust_id', 'is_month_end0', 'is_month_end1']
train = train.replace(np.nan,-1)
test = test.replace(np.nan,-1)
train = train.merge(k1,on=['cust_id'],how='left')
test = test.merge(k1,on=['cust_id'],how='left')

In [ ]:
# columns common 
[i for i in train.columns if i not in test.columns],[i for i in test.columns if i not in train.columns]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# script for validation
from sklearn.metrics import roc_auc_score
def validation(train,test):
    y = train.target.values
    train = train.drop(['target','order_date'],axis=1)
    test = test.drop(['order_date'],axis=1)
    clf = RandomForestClassifier(max_depth=10,criterion='gini',n_estimators=100,n_jobs=-1)
    clf.fit(train,y)
    return clf.predict_proba(test)[:,1]
    

In [ ]:
#0.5836903334386723
k = train.loc[train.order_date > pd.to_datetime('2016-03-20',format= '%Y-%m-%d'),'order_date'].values
t = len(np.unique(k))
print t
met =[]
for i in range(0,t):
    print i
    o = pd.to_datetime('2016-03-20',format= '%Y-%m-%d') + pd.DateOffset(i)
    ind1 = train.loc[train.order_date <= o].index
    ind2 = train.loc[train.order_date > o].index
    X_train = train.iloc[ind1]
    X_test = train.iloc[ind2]
    test_y = X_test.target.values
    X_test = X_test.drop('target',axis=1)
    print X_train.shape,X_test.shape
    pred_test_y = validation(X_train.copy(),X_test.copy())
    e = roc_auc_score(test_y, pred_test_y)
    print (e)
    met.append(e)
print ("mean",np.mean(met))
test.to_csv('7_day_op.csv',index = False)

In [ ]:
met

In [ ]:
clf = RandomForestClassifier(max_depth=8,criterion='gini',n_estimators=350,n_jobs=-1)
clf.fit(train,y)
clf.predict(test)
